# Fetching and Updating Data

In [33]:
# First import the libraries that we need to use
#https://www.cryptodatadownload.com/blog/how-to-download-coinbase-price-data.html
#https://docs.pro.coinbase.com/#get-product-order-book
import pandas as pd
import requests
import json
import os.path
import numpy as np
import os

In [34]:
os.getcwd()

'/Users/alderik/Documents/code/crypto-model-template'

### Fetch Data from Coinbase

In [35]:
def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity=86400'
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume

        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            data.to_csv(f'{os.getcwd()}/newdata/Coinbase_{pair_split[0] + pair_split[1]}_dailydata.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")


# if __name__ == "__main__":
#     # we set which pair we want to retrieve data for
#     pair = "LTC/USD"
#     fetch_daily_data(symbol=pair)

In [36]:
tikkers = ['BTC', 'ETH', 'ADA', 'UNI', 'LTC','LINK','BCH','XLM','WBTC','FIL','AAVE', 'ATOM', 'EOS','XTZ', 'DAI', 'ALGO','DASH','SUSHI','SNX','MKR','GRT','COMP','MATIC','BAT','ZEC','MANA','ETC','UMA','YFI','BNT', 'ZRX','REN','ANKR','OMG','CGLD','CRV', 'LRC','KNC','BAL','SKL','REP','STORJ','CVC','BAND','OXT','NU','NMR','DNT']
names = ['Bitcoin','Ethereum','Cardano','Uniswap','Litecoin','Chainlink','Bitcoin Cash','Stellar Lumens','Wrapped Bitcoin','Filecoin','Aave','Cosmos','EOS','Tezos','Dai','Algorand','Dash','SushiSwap','Synthetic Network Token','Maker', 'The Graph','Compound','Polygon','Basic Attention Token','Zcash','Decentraland','Ethereum Classic','UMA','yearn.finance','Bancor Network Token','Ox','Ren','Ankr','OMG Network','Celo','Curve DAO Token','Loopring','Kyber Network','Balancer','SKALE','Augur','Storj','Civic','Band Protocol','Orchid','NuCypher','Numeraire','district0x']

In [37]:
#fetch data from Coinbase and saves in merge directory. Prints the succesful data pulls
for i in tikkers:
    pair = i+'/USD'
    fetch_daily_data(pair)
    print("Imported ", pair)

Imported  BTC/USD
Imported  ETH/USD
Imported  ADA/USD
Imported  UNI/USD
Imported  LTC/USD
Imported  LINK/USD
Imported  BCH/USD
Imported  XLM/USD
Imported  WBTC/USD
Imported  FIL/USD
Imported  AAVE/USD
Imported  ATOM/USD
Imported  EOS/USD
Imported  XTZ/USD
Imported  DAI/USD
Imported  ALGO/USD
Imported  DASH/USD
Imported  SUSHI/USD
Imported  SNX/USD
Imported  MKR/USD
Imported  GRT/USD
Imported  COMP/USD
Imported  MATIC/USD
Imported  BAT/USD
Imported  ZEC/USD
Imported  MANA/USD
Imported  ETC/USD
Imported  UMA/USD
Imported  YFI/USD
Imported  BNT/USD
Imported  ZRX/USD
Imported  REN/USD
Imported  ANKR/USD
Imported  OMG/USD
Imported  CGLD/USD
Imported  CRV/USD
Imported  LRC/USD
Imported  KNC/USD
Imported  BAL/USD
Imported  SKL/USD
Imported  REP/USD
Imported  STORJ/USD
Did not receieve OK response from Coinbase API
Imported  CVC/USD
Imported  BAND/USD
Imported  OXT/USD
Imported  NU/USD
Imported  NMR/USD
Did not receieve OK response from Coinbase API
Imported  DNT/USD


### Join recently fetched data with existing data

In [38]:
def joining_new_pulled_data(tikkers):
    """
    combines old scraped data with new scraped data
    saves the new df to csv in /combinedata
    """
    for i in tikkers:
        pair = i+'USD'
        orig_path =f'{os.getcwd()}/data/Coinbase_'+pair+'_dailydata.csv'
        new_data_path =f'{os.getcwd()}/newdata/Coinbase_'+pair+'_dailydata.csv'
        combined_path = f'{os.getcwd()}/data/Coinbase_'+pair+'_dailydata.csv'
        if os.path.isfile(orig_path) == True:
            orig = pd.read_csv(orig_path)
            new_data = pd.read_csv(new_data_path)
            orig['date'] = pd.to_datetime(orig['date'])
            new_data['date'] = pd.to_datetime(new_data['date'])
            #check if the new data data is sooner
            if orig.loc[0].date < new_data.loc[0].date:
                print('Adding new values for '+ pair)
                new_rows = new_data[new_data['date'] > orig.loc[0].date]
                full_data = orig.append(new_rows)
                full_data = full_data.sort_values(by= 'date', ascending = False).reset_index()
                full_data = full_data.drop(['index'], axis=1)
                full_data.to_csv(combined_path)
        elif os.path.isfile(new_data_path) == True:
            full_data = pd.read_csv(new_data_path)
            full_data.to_csv(combined_path)
        else:
            print(pair + ' has no data')

In [39]:
joining_new_pulled_data(tikkers)

CVCUSD has no data
DNTUSD has no data


In [40]:
#tikkers minus CVC and DNT becuase those don't have any data
tikkers = ['BTC', 'ETH', 'ADA', 'UNI', 'LTC','LINK','BCH','XLM','WBTC','FIL','AAVE', 'ATOM', 'EOS','XTZ', 'DAI', 'ALGO','DASH','SUSHI','SNX','MKR','GRT','COMP','MATIC','BAT','ZEC','MANA','ETC','UMA','YFI','BNT', 'ZRX','REN','ANKR','OMG','CGLD','CRV', 'LRC','KNC','BAL','SKL','REP','STORJ','BAND','OXT','NU','NMR']

In [41]:
#double checking the start date is the same
for i in tikkers:
    pair = i+'USD'
    temp = pd.read_csv(f'{os.getcwd()}/data/Coinbase_'+pair+'_dailydata.csv', usecols = ['unix', 'low','high','open','close','volume', 'date','vol_fiat'])
    print(temp.iloc[-1]['date'])

2020-08-09
2020-08-09
2021-03-18
2020-09-17
2020-08-09
2020-08-09
2020-08-09
2020-08-09
2020-10-20
2020-12-09
2020-12-15
2020-08-09
2020-08-09
2020-08-09
2020-08-09
2020-08-09
2020-08-09
2021-03-11
2020-12-15
2020-08-09
2020-12-17
2020-08-09
2021-03-11
2021-04-20
2020-12-08
2021-04-20
2020-08-09
2020-09-08
2020-09-15
2020-12-15
2020-08-09
2020-10-06
2021-03-25
2020-08-09
2020-09-01
2021-03-25
2020-09-15
2020-08-09
2020-10-06
2021-03-11
2020-08-09
2021-03-25
2020-08-11
2020-08-09
2020-12-02
2020-08-18
